# SAMPLE for SDT modal parameters
In this notebook we will see how to estimate parameters for [*Sound Design Toolkit*](https://github.com/SkAT-VG/SDT)'s (SDT) `modal` objects

## Setup

### Libraries
Install the `sample` package and its dependencies.
The extra `notebooks` will install dependencies for helper functions such as plots

In [ ]:
from IPython.display import clear_output
import sys
!$sys.executable -m pip install lim-sample[notebooks]==1.2.0
clear_output()

### Download test audio
This is for demonstration purposes, you would want to load your own audio here

In [ ]:
from matplotlib import pyplot as plt
from librosa.display import waveplot, specshow
from IPython.display import Audio as play
import librosa

def resize(w=12, h=6):
  plt.gcf().set_size_inches([w, h])

url = "https://chromaticisobar.gitlab.io/pyaprsi2/audio/front-desk-bells-daniel_simon_original.mp3"
fname = ".testaudio.mp3"
!wget $url -O $fname

x, fs = librosa.load(fname, sr=44100)

!rm -f $fname

clear_output()
waveplot(x, sr=fs, alpha=.5, zorder=100)
plt.grid()
resize()
play(x, rate=fs)

## Apply SAMPLE

### Fit a SAMPLE object
Apply SAMPLE to the audio file

**Hint**: start with a small number of sines (e.g. `sinusoidal_model__max_n_sines=8`) and progressively increase it

In [ ]:
from sample import SAMPLE
from scipy import signal
import numpy as np

sample = SAMPLE(
    sinusoidal_model__max_n_sines=8,
    sinusoidal_model__min_sine_dur=0.2,
).fit(x)
reverse = sample.get_params().get("sinusoidal_model__reverse", False)

### Visualize
Visualize tracked partials, then go back and adjust SAMPLE parameters to better model your audio

In [ ]:
from librosa import stft, amplitude_to_db
from sample import plots

_, ax = plt.subplots(1, 2, sharex=True)
X_db = amplitude_to_db(np.abs(stft(np.array(np.flip(x)) if reverse else x)), ref=np.max)
specshow(X_db, sr=fs, x_axis="time", y_axis="hz", ax=ax[0], cmap="Greys")

plots.sine_tracking_2d(sample.sinusoidal_model, ax=ax)
resize()

### Listen back
You can also additively render audio from the model to get an idea of how SAMPLE has modelled your audio

In [ ]:
x_hat = np.clip(sample.predict(np.arange(x.size) / fs), -1, +1)

waveplot(x_hat, sr=fs, alpha=.5, zorder=100)
plt.grid()
resize()
play(x_hat, rate=fs)

### Compare
You can listen to the original and resynthesized sounds together instereo and plot their STFTs side-by-side

In [ ]:
from librosa import stft, amplitude_to_db

ax = plt.subplot(211)
x_dual = np.array([x, x_hat])
for l, xi in zip(("original", "resynthesis"), x_dual):
    waveplot(xi, sr=fs, alpha=.5, zorder=100, label=l, ax=ax)
plt.grid()
plt.legend()

X_db = amplitude_to_db(np.abs(stft(x)), ref=np.max)
ax = plt.subplot(223, sharex=ax)
specshow(X_db, ax=ax, sr=fs, x_axis="time", y_axis="hz")
ax.set_title("original")

X_hat_db = amplitude_to_db(np.abs(stft(x_hat)), ref=np.max)
ax = plt.subplot(224, sharex=ax, sharey=ax)
specshow(X_hat_db, ax=ax, sr=fs, x_axis="time", y_axis="hz")
ax.set_title("resynthesis")
ax.set_ylim([0, 16000])

resize(12, 12)
play(x_dual, rate=fs)

### Adjust
Now, go back and adjust the parameters.
Some common parameters that you could want to tweak are

- `sinusoidal_model__max_n_sines`: the number of concurrent partials
- `sinusoidal_model__t`: threshold for peak detection (dB)
- `sinusoidal_model__peak_threshold`: threshold for the magnitude intercept (dB at time=0)
- `sinusoidal_model__freq_dev_offset` and `sinusoidal_model__freq_dev_slope`: they control the frequency deviation threshold for the peak continuation. Threshold at frequency $f$ is $$\text{offset}+\text{slope}\cdot f$$

Below, all current parameters are listed

In [ ]:
{
    k: (str(v) if isinstance(v, np.ndarray) else v)
    for k, v in sample.get_params().items()
}

**Hint**:
Some good parameters for this sound could be
```
sinusoidal_model__max_n_sines=64,
sinusoidal_model__peak_threshold=-66,
sinusoidal_model__freq_dev_slope=.005,
sinusoidal_model__min_sine_dur=0.1,
sinusoidal_model__w=signal.blackmanharris(4096),
sinusoidal_model__n=4096,
sinusoidal_model__reverse=True,
```

## Export
Once you are happy with your model, you can export the parameters in a SDT-compliant JSON file

In [ ]:
import json
print(json.dumps(sample.sdt_params_(), indent=2))